In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''
Step 1:
'''

# MNIST dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False, 
                              transform=transforms.ToTensor())




9913344it [00:05, 1862255.99it/s]                             


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



29696it [00:00, 2526246.38it/s]          


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



1649664it [00:00, 1754835.19it/s]                             


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



5120it [00:00, 14628635.20it/s]         

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw




/Users/ardo/miniconda3/envs/pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
'''
Step 2: LeNet5
'''

# Modern LeNet uses this layer for C3
class C3_layer_full(nn.Module):
    def __init__(self):
        super(C3_layer_full, self).__init__()
        self.conv_layer = nn.Conv2d(6, 16, kernel_size=5)

    def forward(self, x):
        return self.conv_layer(x)

# Original LeNet uses this layer for C3
class C3_layer(nn.Module):
    def __init__(self):
        super(C3_layer, self).__init__()
        self.ch_in_3 = [[0, 1, 2],
                        [1, 2, 3],
                        [2, 3, 4],
                        [3, 4, 5],
                        [0, 4, 5],
                        [0, 1, 5]] # filter with 3 subset of input channels
        self.ch_in_4 = [[0, 1, 2, 3],
                        [1, 2, 3, 4],
                        [2, 3, 4, 5],
                        [0, 3, 4, 5],
                        [0, 1, 4, 5],
                        [0, 1, 2, 5],
                        [0, 1, 3, 4],
                        [1, 2, 4, 5],
                        [0, 2, 3, 5]] # filter with 4 subset of input channels
        # put implementation here
        self.conv_layer_ch_in_3 = nn.ModuleList()
        for _ in self.ch_in_3:
                self.conv_layer_ch_in_3.append(nn.Conv2d(3, 1, kernel_size=5))
        self.conv_layer_ch_in_4 = nn.ModuleList()
        for _ in self.ch_in_4:
                self.conv_layer_ch_in_4.append(nn.Conv2d(4, 1, kernel_size=5))
        self.conv_layer_ch_in_6 = nn.Conv2d(6, 1, kernel_size=5)
        

    def forward(self, x):
        # put implementation here
        output = torch.Tensor([]).to(device)
        for i, in_channel in enumerate(self.ch_in_3):
                output = torch.cat((output, self.conv_layer_ch_in_3[i](x[:, in_channel, :, :])), 1)
        for i, in_channel in enumerate(self.ch_in_4):
                output = torch.cat((output, self.conv_layer_ch_in_4[i](x[:, in_channel, :, :])), 1)
        return torch.cat((output, self.conv_layer_ch_in_6(x)), 1)
    


In [4]:
class LeNet(nn.Module) :
    def __init__(self) :
        super(LeNet, self).__init__()
        #padding=2 makes 28x28 image into 32x32
        self.C1_layer = nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=5, padding=2),
                nn.Tanh()
                )
        self.P2_layer = nn.Sequential(
                nn.AvgPool2d(kernel_size=2, stride=2),
                nn.Tanh()
                )
        self.C3_layer = nn.Sequential(
                C3_layer_full(),
                #C3_layer(),
                nn.Tanh()
                )
        self.P4_layer = nn.Sequential(
                nn.AvgPool2d(kernel_size=2, stride=2),
                nn.Tanh()
                )
        self.C5_layer = nn.Sequential(
                nn.Linear(5*5*16, 120),
                nn.Tanh()
                )
        self.F6_layer = nn.Sequential(
                nn.Linear(120, 84),
                nn.Tanh()
                )
        self.F7_layer = nn.Linear(84, 10)
        self.tanh = nn.Tanh()
        
    def forward(self, x) :
        output = self.C1_layer(x)
        output = self.P2_layer(output)
        output = self.C3_layer(output)
        output = self.P4_layer(output)
        output = output.view(-1,5*5*16)
        output = self.C5_layer(output)
        output = self.F6_layer(output)
        output = self.F7_layer(output)
        return output



In [ ]:
# class LeNet_Combination(nn.Module) :
#     def __init__(self) :
#         super(LeNet, self).__init__()
#         #padding=2 makes 28x28 image into 32x32
#         self.model1 = LeNet()
#         self.model1.load_state_dict(torch.load('./trained_model/lenet_pmnist1.pt'))
#         self.model2 = LeNet()
#         self.model2.load_state_dict(torch.load('./trained_model/lenet_pmnist2.pt'))
#         self.model1.to(device)
#         self.model2.to(device)

#         self.C1_layer = nn.Sequential(
#                 CombinationConv2d(1, 6, kernel_size=5, padding=2),
#                 nn.Tanh()
#                 )
#         self.P2_layer = nn.Sequential(
#                 nn.AvgPool2d(kernel_size=2, stride=2),
#                 nn.Tanh()
#                 )
#         self.C3_layer = nn.Sequential(
#                 C3_layer_full(),
#                 #C3_layer(),
#                 nn.Tanh()
#                 )
#         self.P4_layer = nn.Sequential(
#                 nn.AvgPool2d(kernel_size=2, stride=2),
#                 nn.Tanh()
#                 )
#         self.C5_layer = nn.Sequential(
#                 nn.Linear(5*5*16, 120),
#                 nn.Tanh()
#                 )
#         self.F6_layer = nn.Sequential(
#                 nn.Linear(120, 84),
#                 nn.Tanh()
#                 )
#         self.F7_layer = nn.Linear(84, 10)
#         self.tanh = nn.Tanh()
        
#     def forward(self, x) :
#         output = self.C1_layer(x)
#         output = self.P2_layer(output)
#         output = self.C3_layer(output)
#         output = self.P4_layer(output)
#         output = output.view(-1,5*5*16)
#         output = self.C5_layer(output)
#         output = self.F6_layer(output)
#         output = self.F7_layer(output)
#         return output

    



In [17]:
'''
Step 3
'''
model = LeNet().to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# print total number of trainable parameters
param_ct = sum([p.numel() for p in model.parameters()])
print(f"Total number of trainable parameters: {param_ct}")
for n, p in model.named_parameters():
    print(n)


Total number of trainable parameters: 61706
C1_layer.0.weight
C1_layer.0.bias
C3_layer.0.conv_layer.weight
C3_layer.0.conv_layer.bias
C5_layer.0.weight
C5_layer.0.bias
F6_layer.0.weight
F6_layer.0.bias
F7_layer.weight
F7_layer.bias


In [ ]:
'''
Step 4
'''
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

np.random.seed(0)
perm_inds = list(range(28*28))
np.random.shuffle(perm_inds)
#fig = plt.figure(2, figsize=(15, 6))
#fig.suptitle('Correctly-classified Figures', fontsize=16)

import time
start = time.time()
for epoch in range(10) :
    print("{}th epoch starting.".format(epoch))
    for images, labels in train_loader :
        images, labels = images.to(device), labels.to(device)
        plt.imsave('tmp1.png', images[0].cpu().reshape(28,28), cmap = 'gray')
        images = images.reshape(images.shape[0], -1)
        images = images[:, perm_inds].reshape(images.shape[0], 1, 28, 28)
        plt.imsave('tmp2.png',images[0].cpu().reshape(28,28), cmap = 'gray')
        print(0/0)
        optimizer.zero_grad()
        train_loss = loss_function(model(images), labels)
        train_loss.backward()

        optimizer.step()
end = time.time()
print("Time ellapsed in training is: {}".format(end - start))

#torch.save(model.state_dict(), './trained_model/"

'''
Step 5
'''
test_loss, correct, total = 0, 0, 0

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

for images, labels in test_loader :
    images, labels = images.to(device), labels.to(device)

    output = model(images)
    test_loss += loss_function(output, labels).item()

    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(labels.view_as(pred)).sum().item()
    
    total += labels.size(0)
            
print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss /total, correct, total,
        100. * correct / total))